In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window 

In [2]:
spark = SparkSession.builder.master("local[*]").appName("garage").getOrCreate()

customer = spark.read.csv(r"customer.csv", header=True, inferSchema=True)
ser_det = spark.read.csv(r"ser_det.csv", header=True, inferSchema=True)
employee = spark.read.csv(r"employee.csv", header=True, inferSchema=True)
sparepart = spark.read.csv(r"sparepart.csv", header=True, inferSchema=True)
purchase = spark.read.csv(r"purchase.csv", header=True, inferSchema=True)
vendor = spark.read.csv(r"vendors.csv", header=True, inferSchema=True)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/12 07:55:12 WARN Utils: Your hostname, Pavans-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.14 instead (on interface en0)
25/08/12 07:55:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/12 07:55:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
print("customer_table")
customer.show(truncate=False)
print("ser_det_table")
ser_det.show(truncate=False)
print("employee_table")
employee.show(truncate=False)
print("sparepart_table")
sparepart.show(truncate=False)
print("purchase_table")
purchase.show(truncate=False)
print("vendor_table")
vendor.show(truncate=False)



+----+-------------+---------+--------+----------+---------+------+
|cid |cname        |cadd     |contact |creditdays|date     |gender|
+----+-------------+---------+--------+----------+---------+------+
|1001|cyona blake  |NEW YORK |12345684|20        |20-JAN-11|female|
|1002|JOHN SMITH   |NEW JERSI|1341684 |20        |21-FEB-11|male  |
|1003|JORDEN WOOD  |PRAG     |1805184 |20        |22-MAR-11|male  |
|1004|CHRISTANA    |MANHATTON|1125684 |31        |23-APR-13|female|
|1005|TOM HILL     |LONDON   |1239284 |10        |25-JUN-15|male  |
|1006|KAMILA JOSEF |PRAISE   |124568  |9         |28-JUL-11|female|
|1007|ANDRU SYMON  |TEXAS    |125654  |15        |01-APR-16|male  |
|1008|SANJU SAMSUNG|NEW YORK |12346341|4         |20-JAN-16|male  |
+----+-------------+---------+--------+----------+---------+------+

+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|sid |cid |eid |spid|type_veh    |veh_no    |typ_ser       |ser_date 

In [4]:
customer.explain()

== Physical Plan ==
FileScan csv [cid#17,cname#18,cadd#19,contact#20,creditdays#21,date#22,gender#23] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/pavanhalde/Documents/CS/_02_CS/03_Big_Data/Questions_01/cu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<cid:int,cname:string,cadd:string,contact:int,creditdays:int,date:string,gender:string>




In [ ]:
# "Q.1  List all the customers serviced."

serviced = customer.join(ser_det, customer['cid'] == ser_det['cid'], 'inner')
serviced.show(truncate=False)
# customer.join(ser_det, 'cid').show()


+----+------------+---------+--------+----------+---------+------+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|cid |cname       |cadd     |contact |creditdays|date     |gender|sid |cid |eid |spid|type_veh    |veh_no    |typ_ser       |ser_date |qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+------------+---------+--------+----------+---------+------+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|1001|cyona blake |NEW YORK |12345684|20        |20-JAN-11|female|6001|1001|3001|4001|TWO WHEELER |MH15CA3228|TUBE DAMAGED  |02-JAN-11|1.0|250    |250   |35  |50     |0   |335  |
|1002|JOHN SMITH  |NEW JERSI|1341684 |20        |21-FEB-11|male  |6002|1002|3002|4002|TWO WHEELER |MH16U5713 |FULL SERVICING|04-MAR-11|1.0|400    |400   |52  |300    |50  |752  |
|1003|JORDEN WOOD |PRAG     |1805184 |20        |22-MAR-11|male  |6003|1003|3004|4005|TWO WHEELER |MH12PQ

25/07/26 09:35:28 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 969484 ms exceeds timeout 120000 ms
25/07/26 09:35:28 WARN SparkContext: Killing executors is not supported by current scheduler.
25/07/26 09:35:32 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [6]:
# customer.join(ser_det, 'cid').show()

+----+------------+---------+--------+----------+---------+------+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
| cid|       cname|     cadd| contact|creditdays|     date|gender| sid| eid|spid|    type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+------------+---------+--------+----------+---------+------+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|1001| cyona blake| NEW YORK|12345684|        20|20-JAN-11|female|6001|3001|4001| TWO WHEELER|MH15CA3228|  TUBE DAMAGED|02-JAN-11|1.0|    250|   250|  35|     50|   0|  335|
|1002|  JOHN SMITH|NEW JERSI| 1341684|        20|21-FEB-11|  male|6002|3002|4002| TWO WHEELER| MH16U5713|FULL SERVICING|04-MAR-11|1.0|    400|   400|  52|    300|  50|  752|
|1003| JORDEN WOOD|     PRAG| 1805184|        20|22-MAR-11|  male|6003|3004|4005| TWO WHEELER|MH12PQ1313|   CLUTCH WIRE|22-AUG-11|

In [7]:
serviced.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [cid#17], [cid#49], Inner, BuildLeft, false
   :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=312]
   :  +- Filter isnotnull(cid#17)
   :     +- FileScan csv [cid#17,cname#18,cadd#19,contact#20,creditdays#21,date#22,gender#23] Batched: false, DataFilters: [isnotnull(cid#17)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/d:/_02_CS/03_Big_Data/Questions_01/customer.csv], PartitionFilters: [], PushedFilters: [IsNotNull(cid)], ReadSchema: struct<cid:int,cname:string,cadd:string,contact:int,creditdays:int,date:string,gender:string>
   +- Filter isnotnull(cid#49)
      +- FileScan csv [sid#48,cid#49,eid#50,spid#51,type_veh#52,veh_no#53,typ_ser#54,ser_date#55,qty#56,sp_rate#57,sp_amt#58,sp_g#59,ser_amt#60,comm#61,total#62] Batched: false, DataFilters: [isnotnull(cid#49)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/d:/_02_CS/03_Big_D

In [8]:
# "Q.2  Customers who are not serviced."

not_serviced = customer.join(ser_det, customer['cid'] == ser_det['cid'], 'leftanti')
not_serviced.show()

# customer.join(ser_det, 'cid', 'anti').show()

# note that in case of "leftanti" we are getting only the columns from left table.
# and "leftanti" gives uncomman records from the left table.


+----+-------------+--------+--------+----------+---------+------+
| cid|        cname|    cadd| contact|creditdays|     date|gender|
+----+-------------+--------+--------+----------+---------+------+
|1005|     TOM HILL|  LONDON| 1239284|        10|25-JUN-15|  male|
|1008|SANJU SAMSUNG|NEW YORK|12346341|         4|20-JAN-16|  male|
+----+-------------+--------+--------+----------+---------+------+



In [8]:
not_serviced.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [cid#17], [cid#49], LeftAnti, BuildRight, false
   :- FileScan csv [cid#17,cname#18,cadd#19,contact#20,creditdays#21,date#22,gender#23] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/d:/_02_CS/03_Big_Data/Questions_01/customer.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<cid:int,cname:string,cadd:string,contact:int,creditdays:int,date:string,gender:string>
   +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=321]
      +- Filter isnotnull(cid#49)
         +- FileScan csv [cid#49] Batched: false, DataFilters: [isnotnull(cid#49)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/d:/_02_CS/03_Big_Data/Questions_01/ser_det.csv], PartitionFilters: [], PushedFilters: [IsNotNull(cid)], ReadSchema: struct<cid:int>




In [6]:
## "Q.3  Employees who have not received the commission." 

result = ser_det.join(employee,'eid').where(col('comm')==0).select(employee.ename,ser_det.comm)
result.show()
# try if you can eliminate the dublicate values


+-----------+----+
|      ename|comm|
+-----------+----+
|STEVEN KING|   0|
|STEVEN KING|   0|
|   LUIS POP|   0|
+-----------+----+



In [7]:
result.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [ename#74, comm#54]
   +- BroadcastHashJoin [eid#43], [eid#73], Inner, BuildLeft, false
      :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=243]
      :  +- Filter ((isnotnull(comm#54) AND (comm#54 = 0)) AND isnotnull(eid#43))
      :     +- FileScan csv [eid#43,comm#54] Batched: false, DataFilters: [isnotnull(comm#54), (comm#54 = 0), isnotnull(eid#43)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/pavanhalde/Documents/CS/_02_CS/03_Big_Data/Questions_01/se..., PartitionFilters: [], PushedFilters: [IsNotNull(comm), EqualTo(comm,0), IsNotNull(eid)], ReadSchema: struct<eid:int,comm:int>
      +- Filter isnotnull(eid#73)
         +- FileScan csv [eid#73,ename#74] Batched: false, DataFilters: [isnotnull(eid#73)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/pavanhalde/Documents/CS/_02_CS/03_Big_Data/Questions_01/em..., PartitionF

In [8]:
result = (
    ser_det.join(employee, ser_det['eid'] == employee['eid'])
    .where(ser_det['comm'] == 0)
    .select(employee['ename'], ser_det['comm'])
    .distinct()
)

result.show()


+-----------+----+
|      ename|comm|
+-----------+----+
|STEVEN KING|   0|
|   LUIS POP|   0|
+-----------+----+



In [9]:
result.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[ename#74, comm#54], functions=[])
   +- Exchange hashpartitioning(ename#74, comm#54, 200), ENSURE_REQUIREMENTS, [plan_id=430]
      +- HashAggregate(keys=[ename#74, comm#54], functions=[])
         +- Project [ename#74, comm#54]
            +- BroadcastHashJoin [eid#43], [eid#73], Inner, BuildLeft, false
               :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=425]
               :  +- Filter ((isnotnull(comm#54) AND (comm#54 = 0)) AND isnotnull(eid#43))
               :     +- FileScan csv [eid#43,comm#54] Batched: false, DataFilters: [isnotnull(comm#54), (comm#54 = 0), isnotnull(eid#43)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/pavanhalde/Documents/CS/_02_CS/03_Big_Data/Questions_01/se..., PartitionFilters: [], PushedFilters: [IsNotNull(comm), EqualTo(comm,0), IsNotNull(eid)], ReadSchema: struct<eid:int,comm:int>
     

In [10]:

result = (
    ser_det.join(employee, ser_det['eid'] == employee['eid'])
    .where(ser_det['comm'] == 0)
    .select(employee['ename'], ser_det['comm'])
    .dropDuplicates(['ename']) ##########################
)

result.show()


+-----------+----+
|      ename|comm|
+-----------+----+
|   LUIS POP|   0|
|STEVEN KING|   0|
+-----------+----+



In [11]:
result.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[ename#74], functions=[first(comm#54, false)])
   +- Exchange hashpartitioning(ename#74, 200), ENSURE_REQUIREMENTS, [plan_id=616]
      +- HashAggregate(keys=[ename#74], functions=[partial_first(comm#54, false)])
         +- Project [ename#74, comm#54]
            +- BroadcastHashJoin [eid#43], [eid#73], Inner, BuildLeft, false
               :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=611]
               :  +- Filter ((isnotnull(comm#54) AND (comm#54 = 0)) AND isnotnull(eid#43))
               :     +- FileScan csv [eid#43,comm#54] Batched: false, DataFilters: [isnotnull(comm#54), (comm#54 = 0), isnotnull(eid#43)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/pavanhalde/Documents/CS/_02_CS/03_Big_Data/Questions_01/se..., PartitionFilters: [], PushedFilters: [IsNotNull(comm), EqualTo(comm,0), IsNotNull(eid)], ReadSchema: struct<

In [12]:
result = (
    ser_det.join(employee, ser_det['eid'] == employee['eid'])
    .where(ser_det['comm'] == 0)
    .select(employee['ename'], ser_det['comm'])
    .dropDuplicates(['comm']) ##################################
)

result.show()

+-----------+----+
|      ename|comm|
+-----------+----+
|STEVEN KING|   0|
+-----------+----+



In [13]:
result.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortAggregate(key=[comm#54], functions=[first(ename#74, false)])
   +- Sort [comm#54 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(comm#54, 200), ENSURE_REQUIREMENTS, [plan_id=823]
         +- SortAggregate(key=[comm#54], functions=[partial_first(ename#74, false)])
            +- Sort [comm#54 ASC NULLS FIRST], false, 0
               +- Project [ename#74, comm#54]
                  +- BroadcastHashJoin [eid#43], [eid#73], Inner, BuildLeft, false
                     :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=816]
                     :  +- Filter ((isnotnull(comm#54) AND (comm#54 = 0)) AND isnotnull(eid#43))
                     :     +- FileScan csv [eid#43,comm#54] Batched: false, DataFilters: [isnotnull(comm#54), (comm#54 = 0), isnotnull(eid#43)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/pavanhalde/Documents/CS/_02_CS/03_B

In [15]:
result = (
    ser_det.join(employee, ser_det['eid'] == employee['eid'])
    .where(ser_det['comm'] == 0)
    .select(employee['ename'], ser_det['comm'])
    .dropDuplicates() ##################################
)

result.show()
result.explain()

+-----------+----+
|      ename|comm|
+-----------+----+
|STEVEN KING|   0|
|   LUIS POP|   0|
+-----------+----+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[ename#74, comm#54], functions=[])
   +- Exchange hashpartitioning(ename#74, comm#54, 200), ENSURE_REQUIREMENTS, [plan_id=1154]
      +- HashAggregate(keys=[ename#74, comm#54], functions=[])
         +- Project [ename#74, comm#54]
            +- BroadcastHashJoin [eid#43], [eid#73], Inner, BuildLeft, false
               :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=1149]
               :  +- Filter ((isnotnull(comm#54) AND (comm#54 = 0)) AND isnotnull(eid#43))
               :     +- FileScan csv [eid#43,comm#54] Batched: false, DataFilters: [isnotnull(comm#54), (comm#54 = 0), isnotnull(eid#43)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/pavanhalde/Documents/CS/_02_CS/03_Big_Data/Questions_01/se..., PartitionFilte

In [ ]:
# "Q.4  Name the employee who have maximum Commission."

comm_2 = employee.join(ser_det, employee['eid'] == ser_det['eid'], 'inner')
max_comm = comm_2.groupBy("ename").agg(max('comm').alias("max_comm")).orderBy(desc("max_comm")).show(1)

# employee.join(ser_det,'eid').groupBy("ename").agg(max('comm').alias("max_comm")).orderBy(desc("max_comm")).show(1)

+-----------+--------+
|      ename|max_comm|
+-----------+--------+
|STEVEN KING|     150|
+-----------+--------+
only showing top 1 row



In [22]:
employee.show()
ser_det.show()

somm_3 = employee.join(ser_det, "eid").groupBy('eid').agg(sum('comm').alias('sum_commm'))
max_sum_row = somm_3.orderBy(col('sum_commm').desc()).limit(1)
max_sum_row.show()

# this is more correct as it is addingt he sum of the commission for a particualr employee

+----+------------+--------+---------+--------+----+---------+---------+
| eid|       ename|    ejob|     eadd|econtact|esal|     edoj|     edol|
+----+------------+--------+---------+--------+----+---------+---------+
|3001| STEVEN KING| PAINTER| NEW YORK|10367454|1200|01-JAN-08|     null|
|3002|DAVID AUSTIN|  FITTER| MANAHTON|10367434| 110|19-AUG-10|     null|
|3003|BRUCE ERENST|MECHANIC|NEW JERCY|10367264|2200|08-SEP-10|   null\t|
|3004|    LUIS POP|MECHANIC|NEW JERCY|10367264|1700|19-OCT-19|06-SEP-10|
|3005| SHERI GOMES|  FITTER|    PARIS|10327264|1000|19-OCT-09|01-AUG-10|
|3000|JAMES PHILIP|  FITTER|    PARIS|10322264|null|01-JAN-08|     null|
+----+------------+--------+---------+--------+----+---------+---------+

+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|    type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+----+----+----+------------+-------

In [ ]:
# "Q.5  Show employee name and minimum commission amount received by an employee."

comm_3 = ser_det.join(employee, ser_det['eid'] == employee['eid'], 'inner')
min_comm = comm_3.groupBy("ename").agg(min('comm')).show()

# ser_det.join(employee,'eid').groupBy("ename").agg(min('comm')).show()

+------------+---------+
|       ename|min(comm)|
+------------+---------+
|    LUIS POP|        0|
| STEVEN KING|        0|
|DAVID AUSTIN|       50|
+------------+---------+



In [ ]:
# "Q.6  Display the Middle record from any table."      

window_obj = Window.orderBy("cid")
count_01 = customer.count()
middle_rec = customer.withColumn('row_number', row_number().over(window_obj))
middle_pos = (count_01 + 1) // 2
m = middle_rec.filter(col("row_number") == middle_pos).show() # filter can be used instead of where

+----+---------+---------+-------+----------+---------+------+----------+
| cid|    cname|     cadd|contact|creditdays|     date|gender|row_number|
+----+---------+---------+-------+----------+---------+------+----------+
|1004|CHRISTANA|MANHATTON|1125684|        31|23-APR-13|female|         4|
+----+---------+---------+-------+----------+---------+------+----------+



In [ ]:
window_obj = Window.orderBy("cid")
count_01 = customer.count()
middle_rec = customer.withColumn('row_number', row_number().over(window_obj)).show() 

+----+-------------+---------+--------+----------+---------+------+----------+
| cid|        cname|     cadd| contact|creditdays|     date|gender|row_number|
+----+-------------+---------+--------+----------+---------+------+----------+
|1001|  cyona blake| NEW YORK|12345684|        20|20-JAN-11|female|         1|
|1002|   JOHN SMITH|NEW JERSI| 1341684|        20|21-FEB-11|  male|         2|
|1003|  JORDEN WOOD|     PRAG| 1805184|        20|22-MAR-11|  male|         3|
|1004|    CHRISTANA|MANHATTON| 1125684|        31|23-APR-13|female|         4|
|1005|     TOM HILL|   LONDON| 1239284|        10|25-JUN-15|  male|         5|
|1006| KAMILA JOSEF|   PRAISE|  124568|         9|28-JUL-11|female|         6|
|1007|  ANDRU SYMON|    TEXAS|  125654|        15|01-APR-16|  male|         7|
|1008|SANJU SAMSUNG| NEW YORK|12346341|         4|20-JAN-16|  male|         8|
+----+-------------+---------+--------+----------+---------+------+----------+



In [ ]:
# "Q.7  Display last 4 records of any table."

last_4 = customer.orderBy(col("cid").desc()).limit(4).show(truncate=False)

+----+-------------+--------+--------+----------+---------+------+
|cid |cname        |cadd    |contact |creditdays|date     |gender|
+----+-------------+--------+--------+----------+---------+------+
|1008|SANJU SAMSUNG|NEW YORK|12346341|4         |20-JAN-16|male  |
|1007|ANDRU SYMON  |TEXAS   |125654  |15        |01-APR-16|male  |
|1006|KAMILA JOSEF |PRAISE  |124568  |9         |28-JUL-11|female|
|1005|TOM HILL     |LONDON  |1239284 |10        |25-JUN-15|male  |
+----+-------------+--------+--------+----------+---------+------+



In [ ]:
last_4 = customer.orderBy(col("cid").desc()).limit(4).show()

+----+-------------+--------+--------+----------+---------+------+
| cid|        cname|    cadd| contact|creditdays|     date|gender|
+----+-------------+--------+--------+----------+---------+------+
|1008|SANJU SAMSUNG|NEW YORK|12346341|         4|20-JAN-16|  male|
|1007|  ANDRU SYMON|   TEXAS|  125654|        15|01-APR-16|  male|
|1006| KAMILA JOSEF|  PRAISE|  124568|         9|28-JUL-11|female|
|1005|     TOM HILL|  LONDON| 1239284|        10|25-JUN-15|  male|
+----+-------------+--------+--------+----------+---------+------+



In [24]:
# "Q.8  Count the number of records without count function from any table."

win_obj_01 = Window.orderBy("cid")
mid_rec_01 = customer.withColumn('row_number', row_number().over(win_obj_01))
mid_rec_01.select(max('row_number')).show()


+---------------+
|max(row_number)|
+---------------+
|              8|
+---------------+



In [23]:
# "Q.9  Delete duplicate records from 'Ser_det' table on cid.(note Please rollback after execution)."

ser_det.drop_duplicates(['cid']).show()

+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|    type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|6001|1001|3001|4001| TWO WHEELER|MH15CA3228|  TUBE DAMAGED|02-JAN-11|1.0|    250|   250|  35|     50|   0|  335|
|6002|1002|3002|4002| TWO WHEELER| MH16U5713|FULL SERVICING|04-MAR-11|1.0|    400|   400|  52|    300|  50|  752|
|6003|1003|3004|4005| TWO WHEELER|MH12PQ1313|   CLUTCH WIRE|22-AUG-11|1.0|    129|   129|   0|     10|   0|  139|
|6005|1004|3001|4009| TWO WHEELER| MH14PA335|         COLOR|21-OCT-11|2.5|    340|   850| 119|    500| 150| 1469|
|6006|1006|3001|4009| TWO WHEELER| MH12WE334|         COLOR|01-DEC-11|2.5|    340|   850| 119|    500| 150| 1469|
|6007|1007|3001|4001|FOUR WHEELER|MH17BB1345|  TUBE DAMAGED|01-JAN-12|1.0|    250|   250

In [ ]:
# "Q.10 Show the name of Customer who have paid maximum amount."

cust1 = ser_det.join(customer, ser_det["cid"] == customer["cid"], 'inner')
max_total = cust1.groupBy("cname").agg(sum("total")).show(1)

# ser_det.join(customer, "cid").groupBy("cname").agg(sum("total")).show(1)

+----------+----------+
|     cname|sum(total)|
+----------+----------+
|JOHN SMITH|      1504|
+----------+----------+
only showing top 1 row



In [29]:
cust1 = ser_det.join(customer, ser_det["cid"] == customer["cid"], 'inner')
max_total = cust1.groupBy("cname").agg(sum("total").alias('sum_total'))
max_total.orderBy(max_total.sum_total.desc()).show(1)

## this one is more correct as we need to order by first

+----------+---------+
|     cname|sum_total|
+----------+---------+
|JOHN SMITH|     1504|
+----------+---------+
only showing top 1 row



In [30]:
# "Q.11 Display Employees who are not currently working."

not_working = employee.join(ser_det, employee['eid'] == ser_det['eid'], 'leftanti').show()
# not_working.show()

+----+------------+--------+---------+--------+----+---------+---------+
| eid|       ename|    ejob|     eadd|econtact|esal|     edoj|     edol|
+----+------------+--------+---------+--------+----+---------+---------+
|3003|BRUCE ERENST|MECHANIC|NEW JERCY|10367264|2200|08-SEP-10|   null\t|
|3005| SHERI GOMES|  FITTER|    PARIS|10327264|1000|19-OCT-09|01-AUG-10|
|3000|JAMES PHILIP|  FITTER|    PARIS|10322264|null|01-JAN-08|     null|
+----+------------+--------+---------+--------+----+---------+---------+



In [31]:
## "Q.12 How many customers serviced their two wheelers."

cstw = ser_det.join(customer,'cid').groupBy(col('type_veh')).agg(count(col('cid'))).where(col('type_veh')=='TWO WHEELER').show()
# check if you can eliminate the customer that are counted twice

+-----------+----------+
|   type_veh|count(cid)|
+-----------+----------+
|TWO WHEELER|         6|
+-----------+----------+



In [40]:
# "Q.13 List the Purchased Items which are used for Customer Service with Unit of that Item."

ser_det.join(sparepart,"spid").groupBy("spname").count().show(truncate=False)

+-----------------------+-----+
|spname                 |count|
+-----------------------+-----+
|TWO WHEELER CLUTCH WIRE|1    |
|WHITE COLOUR           |2    |
|TWO WHEELER ENGINE OIL |2    |
|TWO WHEELER TUBE       |2    |
+-----------------------+-----+



In [41]:
# "Q.14 Customers who have Colored their vehicles."

ser_det.join(customer, 'cid').select(customer.cname,ser_det.typ_ser).where(col("typ_ser") == 'COLOR').show()

+------------+-------+
|       cname|typ_ser|
+------------+-------+
|   CHRISTANA|  COLOR|
|KAMILA JOSEF|  COLOR|
+------------+-------+



In [ ]:
## "Q.15 Find the annual income of each employee inclusive of Commission"

anual_inc = ser_det.join(employee, ser_det['eid'] == employee['eid'], 'inner')
new = anual_inc.withColumn('annual_income', (employee.esal * 12) + (ser_det.comm * 12))
df = new.select(new.ename, new.annual_income).show()
# try if you can sum the commission a particular employee

In [ ]:
# "Q.16 Vendor Names who provides the engine oil."

provider_01 = vendor.join(purchase,'vid').where(col('spid') == 4002 | 4003)
provider_01.select(provider_01.vname).show()

# vendor.join(purchase, 'vid').where(col('spid') == 4002 | 4003).select('vname').show()

# vendor.join(purchase, 'vid').where(col('spid') == 4002 | 4003).select(vendor.vname).show()

In [ ]:
# "Q.17 Total Cost to purchase the Color and name the color purchased."

sparepart.join(purchase, 'spid').filter((purchase.spid == 4009) | (purchase.spid == 4010) ).select(sparepart.spname,purchase.total).show()

In [42]:
# "Q.18 Purchased Items which are not used in 'Ser_det'."

not_used = purchase.join(ser_det, 'spid', 'anti').show()

+----+----+----+----+------+-------+---------+---------+--------+-------+
|spid| pid| vid|pqty|sprate|  spgst|    pdate|transcost|   total|rcv_eid|
+----+----+----+----+------+-------+---------+---------+--------+-------+
|4004|5003|2003|   8|   680|  761.6|12-JUN-11|       50|  6251.6|   3003|
|4006|5005|2005|  20|   100|  300.0|07-SEP-11|       20|  2320.0|   3003|
|4007|5006|2006|  30|   150|  630.0|11-OCT-11|       60|  5190.0|   3000|
|4003|5007|2001|  20|  5000|14000.0|07-SEP-11|     1000|115000.0|   3000|
+----+----+----+----+------+-------+---------+---------+--------+-------+



In [ ]:
# "Q.19 Spare Parts Not Purchased but existing in Spare part"

not_purchased = sparepart.join(purchase, 'spid', 'anti').show(truncate=False)

In [ ]:
## "Q.20 Calculate the Profit/Loss of the Firm. Consider one month salary of each employee for Calculation."

pl = ser_det.join(employee, 'eid').groupBy('ename').agg(sum(col('esal') - col('total')).alias("profit_or_loss")).show()
# try to show the total sum of profit or loss

In [43]:
# "Q.21 Specify the names of customers who have serviced their vehicles more than one time."

count_cust = customer.join(ser_det, 'cid').groupBy(customer.cname, customer.cid).agg(count(ser_det.cid).alias("serve_count"))
service_count = count_cust.select(count_cust.cname, count_cust.serve_count).where(count_cust.serve_count > 1).show()
# count_cust = customer.join(ser_det, 'cid').groupBy(customer.cname, customer.cid).agg(count(ser_det.cid).alias("serve_count"))
# service_count = count_cust.select(customer.cname, count_cust.serve_count).where(count_cust.serve_count > 1).show()

+----------+-----------+
|     cname|serve_count|
+----------+-----------+
|JOHN SMITH|          2|
+----------+-----------+



In [44]:
# "Q.22 List the Items purchased from vendors locationwise."

items_01 = purchase.join(vendor, purchase["vid"] == vendor["vid"]).join(sparepart,sparepart["spid"] == purchase["spid"])
items_02 = items_01.orderBy(desc("vadd")).select(vendor.vadd, sparepart.spname).show(truncate=False)

# purchase.join(vendor,"vid").join(sparepart,"spid").orderBy(desc("vadd")).select(vendor.vadd,sparepart.spname).show(truncate=False)

+---------+-----------------------+
|vadd     |spname                 |
+---------+-----------------------+
|SATARA   |TWO WHEELER CLUTCH WIRE|
|PUNE     |TWO WHEELER TUBE       |
|PUNE     |FOUR WHEELER ENGINE OIL|
|OSMANABAD|TWO WHEELER CLUTCH WIRE|
|OSMANABAD|WHITE COLOUR           |
|OSMANABAD|TWO WHEELER INDICATORS |
|OSMANABAD|TWO WHEELER CLUTCH WIRE|
|NASHIK   |TWO WHEELER CARBORATOR |
|MUNBAI   |TWO WHEELER ENGINE OIL |
|BULDHANA |TWO WHEELER TAIIL LIGHT|
+---------+-----------------------+



In [45]:
# "Q.23 Display count of two wheeler and four wheeler from ser_det table"

ser_det.groupBy(ser_det.type_veh).agg(count(ser_det.type_veh)).show()

+------------+---------------+
|    type_veh|count(type_veh)|
+------------+---------------+
| TWO WHEELER|              6|
|FOUR WHEELER|              1|
+------------+---------------+



In [46]:
# "Q24 Display name of customers who paid highest SPGST and for which item "

customer.join(ser_det,'cid').orderBy(col('sp_g')).limit(1).show()

+----+-----------+----+-------+----------+---------+------+----+----+----+-----------+----------+-----------+---------+---+-------+------+----+-------+----+-----+
| cid|      cname|cadd|contact|creditdays|     date|gender| sid| eid|spid|   type_veh|    veh_no|    typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+-----------+----+-------+----------+---------+------+----+----+----+-----------+----------+-----------+---------+---+-------+------+----+-------+----+-----+
|1003|JORDEN WOOD|PRAG|1805184|        20|22-MAR-11|  male|6003|3004|4005|TWO WHEELER|MH12PQ1313|CLUTCH WIRE|22-AUG-11|1.0|    129|   129|   0|     10|   0|  139|
+----+-----------+----+-------+----------+---------+------+----+----+----+-----------+----------+-----------+---------+---+-------+------+----+-------+----+-----+



In [47]:
# "Q25 Display vendors name who have charged highest SPGST rate for which item"

ser_det.join(customer, 'cid').join(purchase, "spid").join(vendor,"vid").join(sparepart,'spid').orderBy(col('sp_g')).limit(1).select(vendor.vname,sparepart.spname).show(truncate=False)

+------------+-----------------------+
|vname       |spname                 |
+------------+-----------------------+
|RAM KULKARNI|TWO WHEELER CLUTCH WIRE|
+------------+-----------------------+



In [48]:
# "Q26 list name of item and employee name who have received item "

ser_det.join(employee, "eid").join(sparepart, "spid").select(sparepart.spname,employee.ename).show(truncate=False)

+-----------------------+------------+
|spname                 |ename       |
+-----------------------+------------+
|TWO WHEELER TUBE       |STEVEN KING |
|WHITE COLOUR           |STEVEN KING |
|WHITE COLOUR           |STEVEN KING |
|TWO WHEELER TUBE       |STEVEN KING |
|TWO WHEELER ENGINE OIL |DAVID AUSTIN|
|TWO WHEELER ENGINE OIL |DAVID AUSTIN|
|TWO WHEELER CLUTCH WIRE|LUIS POP    |
+-----------------------+------------+



In [49]:
# Q27 Display the Name and Vehicle Number of Customer who serviced his vehicle, And Name the Item used for Service, 
    # And specify the purchase date of that Item with his vendor and Item Unit and Location, 
    
dv = ser_det.join(customer,'cid').join(sparepart,'spid').join(purchase,'spid').join(vendor,'vid')
dv.select(customer.cname, ser_det.veh_no,sparepart.spname, purchase.pqty,purchase.pdate,vendor.vadd).show(truncate=False)

+------------+----------+-----------------------+----+---------+---------+
|cname       |veh_no    |spname                 |pqty|pdate    |vadd     |
+------------+----------+-----------------------+----+---------+---------+
|cyona blake |MH15CA3228|TWO WHEELER TUBE       |10  |01-JAN-11|PUNE     |
|JOHN SMITH  |MH16U5713 |TWO WHEELER ENGINE OIL |4   |02-MAR-11|MUNBAI   |
|JORDEN WOOD |MH12PQ1313|TWO WHEELER CLUTCH WIRE|1   |15-OCT-11|OSMANABAD|
|JORDEN WOOD |MH12PQ1313|TWO WHEELER CLUTCH WIRE|1   |12-OCT-11|OSMANABAD|
|JORDEN WOOD |MH12PQ1313|TWO WHEELER CLUTCH WIRE|10  |22-AUG-11|SATARA   |
|JOHN SMITH  |MH16U5713 |TWO WHEELER ENGINE OIL |4   |02-MAR-11|MUNBAI   |
|CHRISTANA   |MH14PA335 |WHITE COLOUR           |5   |20-OCT-11|OSMANABAD|
|KAMILA JOSEF|MH12WE334 |WHITE COLOUR           |5   |20-OCT-11|OSMANABAD|
|ANDRU SYMON |MH17BB1345|TWO WHEELER TUBE       |10  |01-JAN-11|PUNE     |
+------------+----------+-----------------------+----+---------+---------+



In [50]:
# And employee Name who serviced the vehicle. for Vehicle NUMBER "MH14PA335".

dv1 = ser_det.join(customer,'cid').join(sparepart,'spid').join(purchase,'spid').join(vendor,'vid')
dv1.select(customer.cname, ser_det.veh_no,sparepart.spname, purchase.pqty,purchase.pdate,vendor.vadd).where(ser_det.veh_no == 'MH14PA335').show(truncate=False)

+---------+---------+------------+----+---------+---------+
|cname    |veh_no   |spname      |pqty|pdate    |vadd     |
+---------+---------+------------+----+---------+---------+
|CHRISTANA|MH14PA335|WHITE COLOUR|5   |20-OCT-11|OSMANABAD|
+---------+---------+------------+----+---------+---------+



In [51]:
# """Q28 who belong this vehicle MH14PA335" Display the customer name """

ser_det.join(customer,"cid").where(col("veh_no")=="MH14PA335").select(customer.cname).show()

+---------+
|    cname|
+---------+
|CHRISTANA|
+---------+



In [52]:
# "Q29 Display the name of customer who belongs to New York and when he /she service their vehicle on which date"

customer.join(ser_det,"cid").where(col("cadd")=="NEW YORK").select(customer.cname,ser_det.ser_date).show()

+-----------+---------+
|      cname| ser_date|
+-----------+---------+
|cyona blake|02-JAN-11|
+-----------+---------+



In [53]:
# "Q 30 from whom we have purchased items having maximum cost?"

purchase.join(vendor,'vid').orderBy(desc('total')).limit(1).select(vendor.vname,purchase.total).show()

+-----------+--------+
|      vname|   total|
+-----------+--------+
|KIRAN PATIL|115000.0|
+-----------+--------+



In [54]:
# "Q31 Display the names of employees who are not working as Mechanic and that employee done services."

employee.join(ser_det,'eid').where(col('ejob')!="MECHANIC").select(employee.ename,ser_det.typ_ser).show()

+------------+--------------+
|       ename|       typ_ser|
+------------+--------------+
| STEVEN KING|  TUBE DAMAGED|
|DAVID AUSTIN|FULL SERVICING|
|DAVID AUSTIN|FULL SERVICING|
| STEVEN KING|         COLOR|
| STEVEN KING|         COLOR|
| STEVEN KING|  TUBE DAMAGED|
+------------+--------------+



In [56]:
# "Q32 Display the various jobs along with total number of employees in each job. The output should contain only those jobs with more than two employees."

no_of_job = employee.groupBy(employee.ejob).agg(count(employee.ejob).alias('count_ejob'))
no_of_job.select(employee.ejob, no_of_job.count_ejob).where(col('count_ejob') > 1).show()

+--------+----------+
|    ejob|count_ejob|
+--------+----------+
|MECHANIC|         2|
|  FITTER|         3|
+--------+----------+



In [57]:
### Q33 Display the details of employees who done service and give them rank according to their no. of services .

step_01 = employee.join(ser_det,'eid')
step_02 = step_01.groupBy(ser_det.eid,employee.ename).agg(count(ser_det.eid).alias('emp_count01'))
win_obj_09 = Window.orderBy(desc(col('emp_count01')))
step_02.withColumn('denr',dense_rank().over(win_obj_09)).show()

+----+------------+-----------+----+
| eid|       ename|emp_count01|denr|
+----+------------+-----------+----+
|3001| STEVEN KING|          4|   1|
|3002|DAVID AUSTIN|          2|   2|
|3004|    LUIS POP|          1|   3|
+----+------------+-----------+----+



In [58]:
# Q 34 Display those employees who are working as Painter and fitter and who provide service and total count of service done by fitter and painter 

service_type = employee.join(ser_det,'eid').groupBy(employee.ename, employee.ejob).agg(count(col('typ_ser')).alias('counting'))
service_type.select(employee.ename, employee.ejob).where((col('ejob') == 'PAINTER') | (col('ejob') == 'FITTER')).show()

+------------+-------+
|       ename|   ejob|
+------------+-------+
| STEVEN KING|PAINTER|
|DAVID AUSTIN| FITTER|
+------------+-------+



In [ ]:
# Q35 Display employee salary and as per highest salary provide Grade to employee 

df_grade = employee.withColumn('Grading', when(col('esal') > 1700, 'Grade A')
                           .when((col('esal') <= 1700) & (col('esal') > 1500), 'Grade B')
                           .when((col('esal') <= 1500) & (col('esal') > 1000), 'Grade C')
                           .when((col('esal') <= 1000) & (col('esal') > 500), 'Grade D')
                           .otherwise('Grade E'))
df_grade.orderBy(desc('esal')).show(truncate=False)

+----+------------+--------+---------+--------+----+---------+---------+-------+
|eid |ename       |ejob    |eadd     |econtact|esal|edoj     |edol     |Grading|
+----+------------+--------+---------+--------+----+---------+---------+-------+
|3000|JAMES PHILIP|FITTER  |PARIS    |10322264|null|01-JAN-08|null     |Grade E|
|3003|BRUCE ERENST|MECHANIC|NEW JERCY|10367264|2200|08-SEP-10|null\t   |Grade A|
|3004|LUIS POP    |MECHANIC|NEW JERCY|10367264|1700|19-OCT-19|06-SEP-10|Grade B|
|3001|STEVEN KING |PAINTER |NEW YORK |10367454|1200|01-JAN-08|null     |Grade C|
|3002|DAVID AUSTIN|FITTER  |MANAHTON |10367434|110 |19-AUG-10|null     |Grade E|
|3005|SHERI GOMES |FITTER  |PARIS    |10327264|1000|19-OCT-09|01-AUG-10|Grade D|
+----+------------+--------+---------+--------+----+---------+---------+-------+



In [ ]:
df_grade = employee.withColumn('Grading', when(col('esal') > 1700, 'Grade A')
                           .when((col('esal') > 1500), 'Grade B')
                           .when((col('esal') > 1000), 'Grade C')
                           .when((col('esal') > 500), 'Grade D')
                           .otherwise('Grade E'))
df_grade.orderBy(desc('esal')).show(truncate=False)

+----+------------+--------+---------+--------+----+---------+---------+-------+
|eid |ename       |ejob    |eadd     |econtact|esal|edoj     |edol     |Grading|
+----+------------+--------+---------+--------+----+---------+---------+-------+
|3000|JAMES PHILIP|FITTER  |PARIS    |10322264|null|01-JAN-08|null     |Grade E|
|3003|BRUCE ERENST|MECHANIC|NEW JERCY|10367264|2200|08-SEP-10|null\t   |Grade A|
|3004|LUIS POP    |MECHANIC|NEW JERCY|10367264|1700|19-OCT-19|06-SEP-10|Grade B|
|3001|STEVEN KING |PAINTER |NEW YORK |10367454|1200|01-JAN-08|null     |Grade C|
|3002|DAVID AUSTIN|FITTER  |MANAHTON |10367434|110 |19-AUG-10|null     |Grade E|
|3005|SHERI GOMES |FITTER  |PARIS    |10327264|1000|19-OCT-09|01-AUG-10|Grade D|
+----+------------+--------+---------+--------+----+---------+---------+-------+



In [59]:
# Q36 display the 4th record of emp table without using group by and rowid

rec_04 = Window.orderBy(asc('eid'))
employee.withColumn('row_number', row_number().over(rec_04)).where(col('row_number') == 4).show()

+----+------------+--------+---------+--------+----+---------+------+----------+
| eid|       ename|    ejob|     eadd|econtact|esal|     edoj|  edol|row_number|
+----+------------+--------+---------+--------+----+---------+------+----------+
|3003|BRUCE ERENST|MECHANIC|NEW JERCY|10367264|2200|08-SEP-10|null\t|         4|
+----+------------+--------+---------+--------+----+---------+------+----------+



In [60]:
# Q37 Provide a commission 100 to employees who are not earning any commission.

commission = ser_det.withColumn('New_Commission', when(col('comm').isNull() | (col('comm') == 0), 100).otherwise(col('comm'))).show(truncate=False)

+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+--------------+
|sid |cid |eid |spid|type_veh    |veh_no    |typ_ser       |ser_date |qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|New_Commission|
+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+--------------+
|6001|1001|3001|4001|TWO WHEELER |MH15CA3228|TUBE DAMAGED  |02-JAN-11|1.0|250    |250   |35  |50     |0   |335  |100           |
|6002|1002|3002|4002|TWO WHEELER |MH16U5713 |FULL SERVICING|04-MAR-11|1.0|400    |400   |52  |300    |50  |752  |50            |
|6003|1003|3004|4005|TWO WHEELER |MH12PQ1313|CLUTCH WIRE   |22-AUG-11|1.0|129    |129   |0   |10     |0   |139  |100           |
|6004|1002|3002|4002|TWO WHEELER |MH16U5713 |FULL SERVICING|05-MAY-11|1.0|400    |400   |52  |300    |50  |752  |50            |
|6005|1004|3001|4009|TWO WHEELER |MH14PA335 |COLOR         |21-OCT-11|2.5|340    |850   |119 |500

In [62]:
# Q38 write a query that totals no. of services for each day and place the results in descending order

ser_det.groupBy(ser_det.ser_date).agg(count(ser_det.ser_date).alias('per day service')).orderBy(desc(col('per day service'))).show()

+---------+---------------+
| ser_date|per day service|
+---------+---------------+
|05-MAY-11|              1|
|22-AUG-11|              1|
|02-JAN-11|              1|
|21-OCT-11|              1|
|04-MAR-11|              1|
|01-JAN-12|              1|
|01-DEC-11|              1|
+---------+---------------+



In [ ]:
# Q39 Display the service details of those customer who belong from same city as that of respective employee. #####

city = ser_det.join(employee,'eid').join(customer, 'cid')
same_city = city.select(customer.cname, customer.cadd, ser_det.veh_no, ser_det.typ_ser, ser_det.type_veh).where(employee.eadd == customer.cadd).show(truncate=False)

In [71]:
# # Q40 write a query join customers table to itself to find all pairs of customers service by a single employee.

# self_1 = customer.alias('c1').join(customer.alias('c2'),'cid','inner')
# self_2 = self_1.select(customer.cname,customer.cadd, ser_det.sid, count(customer.cid)\
#         .over(Window.partitionBy('eid').orderBy('eid')).alias('count__11')).show()

In [68]:
customer.show()
ser_det.show()

+----+-------------+---------+--------+----------+---------+------+
| cid|        cname|     cadd| contact|creditdays|     date|gender|
+----+-------------+---------+--------+----------+---------+------+
|1001|  cyona blake| NEW YORK|12345684|        20|20-JAN-11|female|
|1002|   JOHN SMITH|NEW JERSI| 1341684|        20|21-FEB-11|  male|
|1003|  JORDEN WOOD|     PRAG| 1805184|        20|22-MAR-11|  male|
|1004|    CHRISTANA|MANHATTON| 1125684|        31|23-APR-13|female|
|1005|     TOM HILL|   LONDON| 1239284|        10|25-JUN-15|  male|
|1006| KAMILA JOSEF|   PRAISE|  124568|         9|28-JUL-11|female|
|1007|  ANDRU SYMON|    TEXAS|  125654|        15|01-APR-16|  male|
|1008|SANJU SAMSUNG| NEW YORK|12346341|         4|20-JAN-16|  male|
+----+-------------+---------+--------+----------+---------+------+

+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|    type_veh|    veh_no|       typ_ser| ser_date

In [69]:
customer.join(ser_det, 'cid').groupBy(col('eid')).agg(count(col('cid'))).show()

+----+----------+
| eid|count(cid)|
+----+----------+
|3001|         4|
|3002|         2|
|3004|         1|
+----+----------+



In [74]:
c = customer.join(ser_det, 'cid').groupBy(col('eid')).select('*').show()

## it is compulsary to use agg. when we are using groupBy ????

AttributeError: 'GroupedData' object has no attribute 'select'

In [75]:
# Q41 List each service number follow by name of the customer who made that service

customer.join(ser_det,'cid').select(customer.cname, ser_det.sid).show()

+------------+----+
|       cname| sid|
+------------+----+
| cyona blake|6001|
|  JOHN SMITH|6004|
|  JOHN SMITH|6002|
| JORDEN WOOD|6003|
|   CHRISTANA|6005|
|KAMILA JOSEF|6006|
| ANDRU SYMON|6007|
+------------+----+



In [78]:
# Q42 Write a query to get details of employee and provide rating on basis of maximum services provide by employee.Note (rating should be like A,B,C,D)

emp_abcd = ser_det.join(employee,'eid').groupBy(employee.eid,employee.ename,employee.esal,employee.eadd).agg(count(ser_det.eid).alias('serv_count'))
emp_rating = emp_abcd.withColumn('Rating',when(col('serv_count')>3,'A').when(col('serv_count')>=2,'B').otherwise('C'))
emp_rating.orderBy(asc('Rating')).show()

+----+------------+----+---------+----------+------+
| eid|       ename|esal|     eadd|serv_count|Rating|
+----+------------+----+---------+----------+------+
|3001| STEVEN KING|1200| NEW YORK|         4|     A|
|3002|DAVID AUSTIN| 110| MANAHTON|         2|     B|
|3004|    LUIS POP|1700|NEW JERCY|         1|     C|
+----+------------+----+---------+----------+------+



In [79]:
# Q43 Write a query to get maximum service amount of each customer with their customer details?

customer.join(ser_det,'cid').groupBy(customer.cid,customer.cname,customer.cadd).agg(sum(ser_det.total).alias('cus_amt')).show()

+----+------------+---------+-------+
| cid|       cname|     cadd|cus_amt|
+----+------------+---------+-------+
|1001| cyona blake| NEW YORK|    335|
|1002|  JOHN SMITH|NEW JERSI|   1504|
|1007| ANDRU SYMON|    TEXAS|    435|
|1004|   CHRISTANA|MANHATTON|   1469|
|1006|KAMILA JOSEF|   PRAISE|   1469|
|1003| JORDEN WOOD|     PRAG|    139|
+----+------------+---------+-------+



In [80]:
# Q44 Get the details of customers with his total no of services ?

customer.join(ser_det,'cid').groupBy(customer.cid,customer.cname,customer.cadd).agg(count(ser_det.cid).alias('cus_amt')).show()

+----+------------+---------+-------+
| cid|       cname|     cadd|cus_amt|
+----+------------+---------+-------+
|1001| cyona blake| NEW YORK|      1|
|1002|  JOHN SMITH|NEW JERSI|      2|
|1007| ANDRU SYMON|    TEXAS|      1|
|1004|   CHRISTANA|MANHATTON|      1|
|1006|KAMILA JOSEF|   PRAISE|      1|
|1003| JORDEN WOOD|     PRAG|      1|
+----+------------+---------+-------+



In [81]:
# Q45 From which location sparepart purchased with highest cost ?

purchase.join(vendor, 'vid').join(sparepart,'spid').orderBy(desc('total')).limit(1).select(vendor.vname,vendor.vadd,sparepart.sprate).show()

+-----------+----+------+
|      vname|vadd|sprate|
+-----------+----+------+
|KIRAN PATIL|PUNE|  5000|
+-----------+----+------+



In [82]:
# Q46 Get the details of employee with their service details who has salary is null

employee.join(ser_det,"eid").where(employee.esal=="null").select(employee.ename,ser_det.typ_ser).show() # is correct

+-----+-------+
|ename|typ_ser|
+-----+-------+
+-----+-------+



In [ ]:
# Q47 find the sum of purchase location wise 

purchase.join(vendor, 'vid').groupBy(col('vadd')).agg(sum('total')).show()

In [ ]:
# Q48 write a query sum of purchase amount in word location wise ?

In [83]:
### Q49 Has the customer who has spent the largest amount money has been give highest rating

cplm = customer.join(ser_det,'cid').groupBy(customer.cid,customer.cname,customer.cadd).agg(sum(ser_det.total).alias('cus_amt'))
c_rate01 = Window.orderBy(asc('cus_amt'))
c_rate02 = cplm.withColumn('row_number', row_number().over(c_rate01))
c_rate02.withColumn('CR',when(col('row_number')==1,'A')
                    .when(col('row_number')==2,'B')
                    .when(col('row_number')==3,'C')
                    .when(col('row_number')==4,'D')
                    .when(col('row_number')==5,'E')
                   .otherwise('F')).show()

+----+------------+---------+-------+----------+---+
| cid|       cname|     cadd|cus_amt|row_number| CR|
+----+------------+---------+-------+----------+---+
|1003| JORDEN WOOD|     PRAG|    139|         1|  A|
|1001| cyona blake| NEW YORK|    335|         2|  B|
|1007| ANDRU SYMON|    TEXAS|    435|         3|  C|
|1004|   CHRISTANA|MANHATTON|   1469|         4|  D|
|1006|KAMILA JOSEF|   PRAISE|   1469|         5|  E|
|1002|  JOHN SMITH|NEW JERSI|   1504|         6|  F|
+----+------------+---------+-------+----------+---+



In [84]:
# Q50 select the total amount in service for each customer for which the total is greater than the amount of the largest service amount in the table

ctst = customer.join(ser_det,'cid').groupBy(customer.cid,customer.cname,customer.cadd).agg(sum(ser_det.total).alias('cus_amt'))
ctst.where(col('cus_amt')>max(ser_det.total)).show()

+----+----------+---------+-------+
| cid|     cname|     cadd|cus_amt|
+----+----------+---------+-------+
|1002|JOHN SMITH|NEW JERSI|   1504|
+----+----------+---------+-------+



In [ ]:
# Q51 List the customer name and sparepart name used for their vehicle and vehicle type

opp_01 = ser_det.join(customer,"cid").join(sparepart,"spid")
opp_01.select(customer.cname,sparepart.spname,ser_det.type_veh).show(truncate=False)

In [ ]:
# Q52 Write a query to get spname ,ename,cname quantity ,rate ,service amount for record exist in service table

result01 = ser_det.join(customer,customer["cid"]==ser_det["cid"],"inner").join(employee,employee["eid"]==ser_det["eid"],"inner")\
    .join(sparepart,sparepart["spid"]==ser_det["spid"],"inner")
result01.select(customer.cname,ser_det.qty,ser_det.sp_rate,employee.ename,sparepart.spname).show()

# result_01 = ser_det.join(customer, "cid").join(employee, "eid").join(sparepart, "spid")
# result_01.select(customer.cname, ser_det.qty, ser_det.sp_rate, employee.ename, sparepart.spname).show()

In [ ]:
# Q53 specify the vehicles owners who’s tube damaged.

customer.join(ser_det, "cid").select(customer.cname,ser_det.typ_ser).where(col("typ_ser") == "TUBE DAMAGED").show()

In [ ]:
# Q.54 Specify the details who have taken full service. ####

customer.join(ser_det,"cid").select(customer.cname,ser_det.typ_ser).where(col("typ_ser")=="FULL SERVICING").show()

In [ ]:
# Q.55 Select the employees who have not worked yet and left the job.

employee.join(ser_det,"eid","anti").where(col("edol")!="null").show()

In [86]:
# Q.56 Select employee who have worked first ever.

employee.orderBy(col('edoj')).limit(1).show()

# employee.orderBy(col('ejob')).show(1)

+----+-----------+-------+--------+--------+----+---------+----+
| eid|      ename|   ejob|    eadd|econtact|esal|     edoj|edol|
+----+-----------+-------+--------+--------+----+---------+----+
|3001|STEVEN KING|PAINTER|NEW YORK|10367454|1200|01-JAN-08|null|
+----+-----------+-------+--------+--------+----+---------+----+



In [ ]:
# Q.57 Display all records falling in odd date

ser_da = ser_det.withColumn('ser_date_1',to_date(col('ser_date'),'dd-MMM-yy'))
odd_date = ser_da.filter(dayofmonth('ser_date_1') % 2 != 0)
odd_date.show()

In [ ]:
# Q.58 Display all records falling in even date.

ser_da2 = ser_det.withColumn('ser_date_2', to_date(col('ser_date'), 'dd-MMM-yy'))
even_date = ser_da2.filter(dayofmonth('ser_date_2') % 2 == 0)
even_date.show()

+----+----+----+----+-----------+----------+--------------+---------+---+-------+------+----+-------+----+-----+----------+
| sid| cid| eid|spid|   type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|ser_date_2|
+----+----+----+----+-----------+----------+--------------+---------+---+-------+------+----+-------+----+-----+----------+
|6001|1001|3001|4001|TWO WHEELER|MH15CA3228|  TUBE DAMAGED|02-JAN-11|1.0|    250|   250|  35|     50|   0|  335|2011-01-02|
|6002|1002|3002|4002|TWO WHEELER| MH16U5713|FULL SERVICING|04-MAR-11|1.0|    400|   400|  52|    300|  50|  752|2011-03-04|
|6003|1003|3004|4005|TWO WHEELER|MH12PQ1313|   CLUTCH WIRE|22-AUG-11|1.0|    129|   129|   0|     10|   0|  139|2011-08-22|
+----+----+----+----+-----------+----------+--------------+---------+---+-------+------+----+-------+----+-----+----------+



In [ ]:
# Q.59 Display the vendors whose material is not yet used.

purchase.join(ser_det,"spid").join(vendor,"vid","anti").show()

In [ ]:
# Q.60 Difference between purchase date and used date of spare part.

diffdates_01 = purchase.join(ser_det,'spid')
colnu21 = diffdates_01.withColumn("diffdat",datediff(to_date("ser_date", 'dd-MMM-yy'), to_date("pdate", 'dd-MMM-yy'))).show()#####

+----+----+----+----+------+-----+---------+---------+------+-------+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+-------+
|spid| pid| vid|pqty|sprate|spgst|    pdate|transcost| total|rcv_eid| sid| cid| eid|    type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|diffdat|
+----+----+----+----+------+-----+---------+---------+------+-------+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+-------+
|4001|5001|2001|  10|   250|350.0|01-JAN-11|     1300|4150.0|   3001|6001|1001|3001| TWO WHEELER|MH15CA3228|  TUBE DAMAGED|02-JAN-11|1.0|    250|   250|  35|     50|   0|  335|      1|
|4002|5002|2002|   4|   400|288.0|02-MAR-11|      100|1988.0|   3001|6002|1002|3002| TWO WHEELER| MH16U5713|FULL SERVICING|04-MAR-11|1.0|    400|   400|  52|    300|  50|  752|      2|
|4005|5009|2006|   1|   129|15.48|15-OCT-11|       50|194.48|   3005|6003|1

In [ ]:
from datetime import datetime as dt
s = dt.strptime("04-JAN-11", "%d-%b-%y")
e = dt.strptime("21-OCT-11", "%d-%b-%y")

ser_det.filter(col("ser_date").between(s,e)).show()

#ser_det.filter(col("ser_date").between(lit(to_date('02-JAN-11','dd-mmm-yy')),lit(to_date("21-OCT-11", 'dd-MMM-yy') ))).show()

+---+---+---+----+--------+------+-------+--------+---+-------+------+----+-------+----+-----+
|sid|cid|eid|spid|type_veh|veh_no|typ_ser|ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+---+---+---+----+--------+------+-------+--------+---+-------+------+----+-------+----+-----+
+---+---+---+----+--------+------+-------+--------+---+-------+------+----+-------+----+-----+



+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
| sid| cid| eid|spid|    type_veh|    veh_no|       typ_ser| ser_date|qty|sp_rate|sp_amt|sp_g|ser_amt|comm|total|
+----+----+----+----+------------+----------+--------------+---------+---+-------+------+----+-------+----+-----+
|6001|1001|3001|4001| TWO WHEELER|MH15CA3228|  TUBE DAMAGED|02-JAN-11|1.0|    250|   250|  35|     50|   0|  335|
|6002|1002|3002|4002| TWO WHEELER| MH16U5713|FULL SERVICING|04-MAR-11|1.0|    400|   400|  52|    300|  50|  752|
|6003|1003|3004|4005| TWO WHEELER|MH12PQ1313|   CLUTCH WIRE|22-AUG-11|1.0|    129|   129|   0|     10|   0|  139|
|6004|1002|3002|4002| TWO WHEELER| MH16U5713|FULL SERVICING|05-MAY-11|1.0|    400|   400|  52|    300|  50|  752|
|6005|1004|3001|4009| TWO WHEELER| MH14PA335|         COLOR|21-OCT-11|2.5|    340|   850| 119|    500| 150| 1469|
|6006|1006|3001|4009| TWO WHEELER| MH12WE334|         COLOR|01-DEC-11|2.5|    340|   850